In [1]:
import sys
import os
import importlib
import time
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer #计算tfidf
from sklearn.feature_extraction.text import CountVectorizer  #计算df
from sklearn.feature_extraction.text import TfidfVectorizer  #“一步到位”
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib # to save
from sklearn import metrics

id_num = []
word_article = []
words_article = []
label = []
X_train = []
y_train = []
X_test = []
y_test = []

load_start = time.time()
importlib.reload(sys)
with open('train_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()[1:]
    random.shuffle(lines) #注意用法
    for line in lines[1:100000]:
        line = line.split(',')
        X_train.append(line[2])
        y_train.append(line[3])
    for line in lines[100000:]: #划分出测试集
        line = line.split(',')
        X_test.append(line[2])
        y_test.append(line[3])
print("Time consumption on loading:",time.time() - load_start)

train_start = time.time()
vect = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, norm='l2', use_idf=1, smooth_idf=1, sublinear_tf=1)
vect.fit(X_train) # 或者直接X_train = vect.fit_transform(X_train)
X_train = vect.transform(X_train)
print(X_train.shape)
# X_train = preprocessing.normalize(X_train, norm='l2', axis=1, copy=True) #变化之后是...一样的
# ss=StandardScaler(with_mean=False).fit(X_train)
# X_train=ss.transform(X_train)
# X_train = preprocessing.scale(X_train, with_mean=False) #标准化
# X_train = X_train.toarray() #由稀疏矩阵变成ndarray类型的完整矩阵。。？
# pca = PCA(n_components = 10000)
# print(X_train.shape)
# X_train = pca.fit_transform(X_train)
# print(X_train.shape)
print("Time consumption on training1:",time.time() - train_start)

clf = svm.LinearSVC(C=0.8, loss='squared_hinge', intercept_scaling=1, tol=0.0002, multi_class='ovr')
# clf = svm.SVC(kernel='linear', C=1, )
# clf = SGDClassifier(loss="modified_huber", )#penalty=”elasticnet”: L2和L1的convex组合; (1 - l1_ratio) * L2 + l1_ratio * L1
# parameters = {#'loss':('squared_hinge', 'hinge'),
#               #'alpha':[0.00004, 0.00001, 0.00002],
#               #'penalty':['elasticnet','l2']
#                 'tot':[0.00005, 0.0001, 0.0002]
# }
# grid_search = GridSearchCV(clf, parameters, n_jobs=-1, cv=5)
# grid_search.fit(X_train, y_train)
# best_parameters = dict(grid_search.best_estimator_.get_params())#get实例中的最优参数
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name])) #输出参数结果
clf.fit(X_train, y_train)
scores = cross_val_score(clf, X_train, y_train , cv=5)
# print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# joblib.dump(clf, 'lineSVM_20000__.m')
print("Time consumption on training1+2:",time.time() - train_start)


test_start = time.time()
# clf = joblib.load('lineSVM_20000__.m')
prediction = clf.predict(X_train)
print(metrics.classification_report(y_train,prediction))
X_test = vect.transform(X_test)
# X_test=ss.transform(X_test)
# X_test = X_test.toarray()
# X_test = preprocessing.normalize(X_test, norm='l2', axis=1, copy=True) #正则化
# X_test = preprocessing.scale(X_test, with_mean=False) 
print(X_test.shape)
prediction = clf.predict(X_test) 
print(metrics.classification_report(y_test,prediction))
# print(metrics.confusion_matrix(y_test,prediction))
print("Time consumption on testing:",time.time() - test_start)

Time consumption on loading: 2.872018814086914
(99999, 2768408)
Time consumption on training1: 384.3682017326355
Accuracy: 0.79 (+/- 0.01)
Time consumption on training1+2: 1408.990906715393
             precision    recall  f1-score   support

         1
       0.98      0.97      0.97      5262
        10
       0.99      0.99      0.99      4846
        11
       0.98      0.99      0.99      3500
        12
       0.99      0.99      0.99      5212
        13
       0.98      0.99      0.99      7712
        14
       0.99      0.99      0.99      6601
        15
       1.00      1.00      1.00      7322
        16
       0.98      0.98      0.98      3147
        17
       0.98      0.99      0.98      3021
        18
       0.99      1.00      1.00      6902
        19
       0.98      0.98      0.98      5427
         2
       0.99      0.99      0.99      2840
         3
       0.99      1.00      1.00      8143
         4
       0.99      0.99      0.99      3734
         5
   